In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
import random

In [17]:
subject = 1
runs = [3]
tmin, tmax = -1.0, 4.0
event_id = dict(rest=1,left=2, right=3)


In [89]:
tmin, tmax = -1.0, 4.0
event_id = dict(rest=1,left=2, right=3)
runs = [3]

train_data = open("PhysionetTrain.csv", "a")
test_data = open("PhysionetTest.csv", "a")
val_data = open("PhysionetVal.csv", "a")

train_data.write('paciente;'+'medicao;'+'eletrodo;'+'label')
val_data.write('paciente;'+'medicao;'+'eletrodo;'+'label')
test_data.write('paciente;'+'medicao;'+'eletrodo;'+'label')


for idx in range(800):
    train_data.write(';leitura'+str(idx))
    val_data.write(';leitura'+str(idx))
    test_data.write(';leitura'+str(idx))

train_data.write('\n')
val_data.write('\n')
test_data.write('\n')

teste = open("teste2.csv", "a")
for subject in range(1,2):
    raw_fnames = eegbci.load_data(subject, runs)
    raw = concatenate_raws([read_raw_edf(f, preload=True) for f in raw_fnames])
    eegbci.standardize(raw)  # set channel names
    montage = make_standard_montage("standard_1005")
    raw.set_montage(montage)
    raw.filter(2.0, 8.0, fir_design="firwin", skip_by_annotation="edge")
    events, _ = events_from_annotations(raw, event_id='auto')
    picks = pick_types(raw.info, meg=False, eeg=True, stim=False, eog=False, exclude="bads")
    channels = raw.ch_names
    channels = [x.replace('.','').upper() for x in channels]
    epochs = Epochs(
        raw,
        events,
        event_id,
        tmin,
        tmax,
        proj=True,
        picks=picks,
        baseline=None,
        preload=True,
    )
    epochs_data = epochs.get_data()
    labels = epochs.events[:, -1]
    flag = True
    random_epochs = random.sample(range(0, 29), 29)
    epochs_val = random_epochs[0:4]
    epochs_test = random_epochs[4:8]
    for epoch in range(len(epochs_data)):
        label = labels[epoch]
        if(label == 1):
            if(flag == True):
                flag = False
            else:
                flag = True
        
        if(label != 1 or (label == 1 and flag)):
            if(epoch in epochs_val):
                for eletrode in range(len(epochs_data[epoch])):
                    val_data.write(str(subject)+';'+str(epoch+1)+';'+channels[eletrode]+';'+str(label))
                    sign = epochs_data[epoch][eletrode][:800]
                    
                    for leitura in range(len(sign)):
                        val_data.write(';'+str(sign[leitura]))
                    val_data.write('\n')
            elif(epoch in epochs_test):
                for eletrode in range(len(epochs_data[epoch])):
                    test_data.write(str(subject)+';'+str(epoch+1)+';'+channels[eletrode]+';'+str(label))
                    sign = epochs_data[epoch][eletrode][:800]
                    
                    for leitura in range(len(sign)):
                        test_data.write(';'+str(sign[leitura]))
                    test_data.write('\n')
            else:
                for eletrode in range(len(epochs_data[epoch])):
                    train_data.write(str(subject)+';'+str(epoch+1)+';'+channels[eletrode]+';'+str(label))
                    sign = epochs_data[epoch][eletrode][:800]
                    
                    for leitura in range(len(sign)):
                        train_data.write(';'+str(sign[leitura]))
                    train_data.write('\n')
train_data.close()
val_data.close()
test_data.close()
        


Extracting EDF parameters from d:\Dev\GitHub\tcc-brain\tcc-brain\dataset\eeg-motor-movementimagery-dataset\files\MNE-eegbci-data\files\eegmmidb\1.0.0\S001\S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 8 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 8.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 9.00 Hz)
- Filter length: 265 samples (1.656 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished


Used Annotations descriptions: ['T0', 'T1', 'T2']
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 801 original time points ...
1 bad epochs dropped


In [40]:
len(epochs_data)

29

In [ ]:
raw.ch_names

In [58]:
vetor = [1,2,3,4,5,6,7,8]

print(vetor[:3])

[1, 2, 3]


In [68]:


epochs_train = random_epochs[8:]


In [69]:
epochs_val

[12, 0, 24, 22]

In [70]:
epochs_test

[28, 21, 4, 2]

In [71]:
epochs_train

[14, 9, 7, 15, 16, 20, 1, 26, 19, 18, 23, 8, 13, 5, 6, 17, 10, 3, 11, 27, 25]

In [79]:
train_dataset = pd.read_csv('PhysionetTrain.csv', delimiter=';')
test_dataset = pd.read_csv('PhysionetTest.csv', delimiter=';')
val_dataset = pd.read_csv('PhysionetVal.csv', delimiter=';')

In [85]:
print(train_dataset['label'].value_counts()[3])

39808


In [86]:
print(test_dataset['label'].value_counts()[2],test_dataset['label'].value_counts()[3])

7488 5696


In [ ]:
print(val_dataset['label'].value_counts()[1],val_dataset['label'].value_counts()[2],val_dataset['label'].value_counts()[3])